# 3D Data Management - Export Operations

High-speed, concurrent export system for client delivery.
Reads staged tasks and delivers files to client folders with absolute precision.

**Mantra: Stability at hyperspeed**

## Prerequisites
- Google account with access to the 3D Data Management sheet
- Staging folder access (read-only)
- Client folder access (write-only)
- Tasks with export_status='staged'

In [ ]:
# Cell 1: Authentication & Setup - Thread-Safe Services
import os
import re
import time
import random
import threading
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock

# Google API imports
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd
import google.auth

# Sheet ID - Update with your actual Sheet ID
SHEET_ID = "1HmDdq5g0Zk7d7Uodbh7fIUFq5XiONDZyvrBBZQgNK0w"

# Global thread-safe credentials
_global_credentials = None
thread_local = threading.local()

def get_credentials():
    """Get shared credentials for all threads"""
    global _global_credentials
    if _global_credentials is None:
        _global_credentials, _ = google.auth.default()
    return _global_credentials

def get_thread_safe_drive_service():
    """Thread-safe Drive service with automatic recreation"""
    if not hasattr(thread_local, 'drive_service'):
        credentials = get_credentials()
        thread_local.drive_service = build('drive', 'v3', credentials=credentials)
    return thread_local.drive_service

def get_thread_safe_sheets_service():
    """Thread-safe Sheets service with automatic recreation"""
    if not hasattr(thread_local, 'sheets_service'):
        credentials = get_credentials()
        thread_local.sheets_service = build('sheets', 'v4', credentials=credentials)
    return thread_local.sheets_service

def reset_thread_services():
    """Reset services on SSL errors"""
    for attr in ['drive_service', 'sheets_service']:
        if hasattr(thread_local, attr):
            delattr(thread_local, attr)

# Initialize authentication
print("🔐 Authenticating with Google...")
auth.authenticate_user()
_global_credentials = get_credentials()

# Test services
try:
    drive_service = get_thread_safe_drive_service()
    sheets_service = get_thread_safe_sheets_service()
    
    # Quick auth test
    about = drive_service.about().get(fields='user').execute()
    user_email = about.get('user', {}).get('emailAddress', 'Unknown')
    
    print(f"✅ Authentication successful: {user_email}")
    print(f"📋 Using Sheet ID: {SHEET_ID}")
    print("🚀 Ready for export operations")
    
except Exception as e:
    print(f"❌ Authentication failed: {e}")
    print("💡 Try: Runtime → Restart Runtime, then re-run")

In [ ]:
# Cell 2: Configuration & Task Reader - Pure and Simple

class ConfigReader:
    """Reads configuration from sheet .env tab"""
    
    def __init__(self, sheet_id):
        self.sheet_id = sheet_id
        self.config = {}
    
    def load_config(self):
        """Load environment variables from .env sheet tab"""
        sheets_service = get_thread_safe_sheets_service()
        
        result = sheets_service.spreadsheets().values().get(
            spreadsheetId=self.sheet_id,
            range='.env!A:B'
        ).execute()
        
        for row in result.get('values', []):
            if len(row) >= 2 and row[0] and row[1]:
                self.config[row[0].strip()] = row[1].strip()
        
        # Validate required staging folder
        if 'STAGING_FOLDER_ID' not in self.config:
            raise ValueError("Missing STAGING_FOLDER_ID in .env configuration")
        
        print(f"⚙️ Config loaded: {len(self.config)} variables")
        return self.config
    
    def get(self, key, default=None):
        return self.config.get(key, default)


class StagedTaskReader:
    """Reads staged tasks ready for export"""
    
    def __init__(self, sheet_id):
        self.sheet_id = sheet_id
    
    def get_export_batches(self):
        """Get available export batches with staged tasks"""
        df = self._read_tasks_sheet()
        
        # Filter for staged tasks only
        staged_tasks = df[df['Export Status'].str.lower() == 'staged']
        
        if staged_tasks.empty:
            return []
        
        # Group by export batch
        batches = staged_tasks.groupby('Export Batch').agg({
            'Task ID': 'count',
            'Export Time': 'first'
        }).rename(columns={'Task ID': 'task_count'}).reset_index()
        
        batches = batches.sort_values('Export Time', ascending=False)
        
        return [{
            'batch_id': row['Export Batch'],
            'task_count': row['task_count'],
            'created': row['Export Time']
        } for _, row in batches.iterrows()]
    
    def get_staged_tasks(self, export_batch_id):
        """Get all staged tasks for a specific export batch"""
        df = self._read_tasks_sheet()
        
        # Filter for specific batch and staged status
        filter_mask = (
            (df['Export Batch'] == export_batch_id) & 
            (df['Export Status'].str.lower() == 'staged')
        )
        
        staged_tasks = df[filter_mask]
        
        return [{
            'task_id': row['Task ID'],
            'folder_name': row['Folder Name'],
            'staging_folder_link': row['Production Folder'],  # Staging folder location
            'batch_id': row['Batch ID'],
            'export_batch_id': row['Export Batch']
        } for _, row in staged_tasks.iterrows()]
    
    def _read_tasks_sheet(self):
        """Read and parse the main Tasks sheet"""
        sheets_service = get_thread_safe_sheets_service()
        
        result = sheets_service.spreadsheets().values().get(
            spreadsheetId=self.sheet_id,
            range='Tasks!A:AC'
        ).execute()
        
        values = result.get('values', [])
        if not values:
            return pd.DataFrame()
        
        headers = values[0]
        data_rows = values[1:]
        
        # Pad rows to match header length
        max_cols = len(headers)
        padded_rows = [row + [''] * (max_cols - len(row)) for row in data_rows]
        
        return pd.DataFrame(padded_rows, columns=headers)


print("📖 Configuration and Task Reader ready")

In [ ]:
# Cell 3: Export Operations - High-Speed File Delivery with Shared Drive Support

class ExportOperations:
    """Core export engine - moves files from staging to client with precision"""
    
    def __init__(self, staging_folder_id):
        self.staging_folder_id = staging_folder_id
        self.progress_lock = Lock()
        self.completed_count = 0
    
    def validate_client_folder(self, client_folder_id):
        """Validate client folder access - FIXED for Shared Drives"""
        try:
            drive_service = get_thread_safe_drive_service()
            
            # Try regular Drive first, then Shared Drive if it fails
            folder = None
            try:
                # Regular Drive API call
                folder = drive_service.files().get(fileId=client_folder_id).execute()
            except Exception as regular_error:
                # If regular fails, try with Shared Drive support
                try:
                    folder = drive_service.files().get(
                        fileId=client_folder_id,
                        supportsAllDrives=True,
                        supportsTeamDrives=True
                    ).execute()
                    print("✅ Found folder in Shared Drive")
                except Exception as shared_error:
                    return {
                        'valid': False,
                        'error': f'Cannot access folder: {str(regular_error)}'
                    }
            
            # Verify it's actually a folder
            if folder.get('mimeType') != 'application/vnd.google-apps.folder':
                return {
                    'valid': False,
                    'error': 'Provided ID is not a folder'
                }
            
            # Test if we can list contents (SAFE read-only check with Shared Drive support)
            try:
                drive_service.files().list(
                    q=f"'{client_folder_id}' in parents",
                    pageSize=1,
                    fields='files(id,name)',
                    supportsAllDrives=True,
                    includeItemsFromAllDrives=True,
                    supportsTeamDrives=True
                ).execute()
            except Exception as list_error:
                return {
                    'valid': False,
                    'error': f'Cannot list folder contents: {str(list_error)}'
                }
            
            return {
                'valid': True,
                'name': folder.get('name', 'Unknown'),
                'id': client_folder_id,
                'is_shared_drive': 'teamDriveId' in folder or 'driveId' in folder
            }
            
        except Exception as e:
            error_msg = str(e)
            
            # Parse common error messages
            if 'not found' in error_msg.lower():
                return {
                    'valid': False,
                    'error': 'Folder not found. For Shared Drives, ensure you have access and use the folder ID (not URL)'
                }
            elif 'permission' in error_msg.lower() or 'forbidden' in error_msg.lower():
                return {
                    'valid': False,
                    'error': 'Insufficient permissions. For Shared Drives, you need at least "Contributor" access'
                }
            else:
                return {
                    'valid': False,
                    'error': f'Access validation failed: {error_msg}'
                }
    
    def export_tasks_concurrent(self, tasks, client_folder_id, max_workers=3):
        """Export tasks with safe concurrency"""
        self.completed_count = 0
        results = []
        
        chunk_size = 8  # Optimal chunk size for stability
        actual_workers = min(max_workers, 3, len(tasks))
        
        print(f"🚀 Starting concurrent export: {len(tasks)} tasks")
        print(f"⚡ Mode: {actual_workers} workers, chunks of {chunk_size}")
        
        total_start = time.time()
        
        # Process in chunks for stability
        for chunk_start in range(0, len(tasks), chunk_size):
            chunk_end = min(chunk_start + chunk_size, len(tasks))
            chunk_tasks = tasks[chunk_start:chunk_end]
            
            print(f"\n📦 Chunk {chunk_start//chunk_size + 1}: tasks {chunk_start+1}-{chunk_end}")
            
            with ThreadPoolExecutor(max_workers=actual_workers) as executor:
                futures = {
                    executor.submit(self._export_single_task, task, client_folder_id): task
                    for task in chunk_tasks
                }
                
                for future in as_completed(futures):
                    result = future.result()
                    results.append(result)
            
            # Brief pause between chunks
            if chunk_end < len(tasks):
                time.sleep(2)
        
        duration = time.time() - total_start
        successful = sum(1 for r in results if r['success'])
        
        print(f"\n🎯 Export Complete: {successful}/{len(tasks)} in {duration:.1f}s")
        return results
    
    def _export_single_task(self, task, client_folder_id):
        """Export single task with automatic retry"""
        max_retries = 3
        base_delay = 1
        
        for attempt in range(max_retries):
            try:
                return self._perform_task_export(task, client_folder_id)
                
            except Exception as e:
                if attempt < max_retries - 1:
                    # Reset services on network errors
                    if any(keyword in str(e).lower() for keyword in ['ssl', 'connection', 'timeout']):
                        reset_thread_services()
                    
                    delay = base_delay * (2 ** attempt) + random.uniform(0, 1)
                    time.sleep(delay)
                    continue
                else:
                    return {
                        'task_id': task['task_id'],
                        'folder_name': task['folder_name'],
                        'success': False,
                        'error': str(e)
                    }
    
    def _perform_task_export(self, task, client_folder_id):
        """Perform the actual file export for one task - FIXED for Shared Drives"""
        start_time = time.time()
        drive_service = get_thread_safe_drive_service()
        
        # Find staging folder for this task - ADD Shared Drive support
        staging_folders = drive_service.files().list(
            q=f"'{self.staging_folder_id}' in parents and name contains '{task['export_batch_id']}' and mimeType='application/vnd.google-apps.folder'",
            fields='files(id,name)',
            supportsAllDrives=True,
            includeItemsFromAllDrives=True,
            supportsTeamDrives=True
        ).execute().get('files', [])
        
        if not staging_folders:
            raise Exception(f"Staging batch folder not found for {task['export_batch_id']}")
        
        batch_folder_id = staging_folders[0]['id']
        
        # Find task folder within batch - ADD Shared Drive support
        task_folders = drive_service.files().list(
            q=f"'{batch_folder_id}' in parents and name='{task['folder_name']}' and mimeType='application/vnd.google-apps.folder'",
            fields='files(id,name)',
            supportsAllDrives=True,
            includeItemsFromAllDrives=True,
            supportsTeamDrives=True
        ).execute().get('files', [])
        
        if not task_folders:
            raise Exception(f"Task folder '{task['folder_name']}' not found in staging")
        
        staging_task_folder_id = task_folders[0]['id']
        
        # Find or create client task folder
        client_task_folder_id = self._get_or_create_client_folder(
            client_folder_id, task['folder_name']
        )
        
        # Copy files with intelligent filtering
        copied_count = self._copy_production_files(
            staging_task_folder_id, 
            client_task_folder_id,
            task['folder_name']
        )
        
        duration = time.time() - start_time
        
        with self.progress_lock:
            self.completed_count += 1
            print(f"✅ ({self.completed_count}) {task['folder_name']}: {copied_count} files in {duration:.1f}s")
        
        return {
            'task_id': task['task_id'],
            'folder_name': task['folder_name'],
            'success': True,
            'file_count': copied_count,
            'duration': duration
        }
    
    def _get_or_create_client_folder(self, client_folder_id, folder_name):
        """Get existing or create new task folder in client drive - FIXED for Shared Drives"""
        drive_service = get_thread_safe_drive_service()
        
        # Check if folder already exists - ADD Shared Drive support
        existing = drive_service.files().list(
            q=f"'{client_folder_id}' in parents and name='{folder_name}' and mimeType='application/vnd.google-apps.folder'",
            fields='files(id,name)',
            supportsAllDrives=True,
            includeItemsFromAllDrives=True,
            supportsTeamDrives=True
        ).execute().get('files', [])
        
        if existing:
            return existing[0]['id']
        
        # Create new folder (ADD-ONLY operation) - ADD Shared Drive support
        folder = drive_service.files().create(
            body={
                'name': folder_name,
                'parents': [client_folder_id],
                'mimeType': 'application/vnd.google-apps.folder'
            },
            supportsAllDrives=True,
            supportsTeamDrives=True
        ).execute()
        
        return folder['id']
    
    def _copy_production_files(self, source_folder_id, target_folder_id, folder_name):
        """Copy only production files, excluding original inputs - FIXED for Shared Drives"""
        drive_service = get_thread_safe_drive_service()
        
        # Get all files in staging folder - ADD Shared Drive support
        files = drive_service.files().list(
            q=f"'{source_folder_id}' in parents and trashed=false",
            fields='files(id,name,mimeType)',
            supportsAllDrives=True,
            includeItemsFromAllDrives=True,
            supportsTeamDrives=True
        ).execute().get('files', [])
        
        # Filter files - exclude original inputs and v0 files
        exclude_files = ['image.jpg', 'img_mask.jpg', 'mask.jpg']
        production_files = [
            f for f in files 
            if f['mimeType'] != 'application/vnd.google-apps.folder'
            and f['name'] not in exclude_files
            and not re.search(r'_v0\.', f['name'])  # Exclude version 0 files
        ]
        
        copied_count = 0
        
        for file in production_files:
            # Check if file already exists in client folder (SAFE check) - ADD Shared Drive support
            existing = drive_service.files().list(
                q=f"'{target_folder_id}' in parents and name='{file['name']}'",
                fields='files(id)',
                supportsAllDrives=True,
                includeItemsFromAllDrives=True,
                supportsTeamDrives=True
            ).execute().get('files', [])
            
            if not existing:  # Only copy if doesn't exist (ADD-ONLY) - ADD Shared Drive support
                drive_service.files().copy(
                    fileId=file['id'],
                    body={
                        'name': file['name'],
                        'parents': [target_folder_id]
                    },
                    supportsAllDrives=True,
                    supportsTeamDrives=True
                ).execute()
                copied_count += 1
        
        return copied_count


print("🚀 Export Operations engine ready (SAFE: ADD-ONLY operations + Shared Drive support)")

In [ ]:
# Cell 4: Status Updater - Sheet Tracking

class ExportStatusUpdater:
    """Updates sheet with export delivery status"""
    
    def __init__(self, sheet_id):
        self.sheet_id = sheet_id
    
    def update_delivery_status(self, export_results):
        """Update export status for all processed tasks"""
        sheets_service = get_thread_safe_sheets_service()
        
        # Get current sheet data - Read full width to AC
        result = sheets_service.spreadsheets().values().get(
            spreadsheetId=self.sheet_id,
            range='Tasks!A:AC'
        ).execute()
        
        values = result.get('values', [])
        if not values:
            return
        
        headers = values[0]
        
        # Find column indices
        task_id_col = self._find_column_index(headers, 'Task ID')
        export_status_col = self._find_column_index(headers, 'Export Status')
        
        if task_id_col == -1 or export_status_col == -1:
            print("⚠️ Required columns not found in sheet")
            return
        
        # Build batch updates
        batch_updates = []
        
        for export_result in export_results:
            task_id = export_result['task_id']
            new_status = 'delivered' if export_result['success'] else 'delivery_failed'
            
            # Find task row
            row_index = self._find_task_row(values, task_id, task_id_col)
            if row_index > 0:
                # Convert column index to Excel column letter
                col_letter = self._index_to_column_letter(export_status_col)
                
                batch_updates.append({
                    'range': f'Tasks!{col_letter}{row_index}',
                    'values': [[new_status]]
                })
        
        # Execute batch update
        if batch_updates:
            sheets_service.spreadsheets().values().batchUpdate(
                spreadsheetId=self.sheet_id,
                body={
                    'valueInputOption': 'RAW',
                    'data': batch_updates
                }
            ).execute()
            
            successful = sum(1 for r in export_results if r['success'])
            print(f"📝 Sheet updated: {successful} delivered, {len(export_results) - successful} failed")
    
    def _find_column_index(self, headers, column_name):
        """Find column index by name"""
        try:
            return headers.index(column_name)
        except ValueError:
            return -1
    
    def _find_task_row(self, values, task_id, task_id_col):
        """Find row index for specific task ID"""
        for i, row in enumerate(values[1:], start=2):  # Start at row 2 (1-indexed)
            if len(row) > task_id_col and row[task_id_col] == task_id:
                return i
        return -1
    
    def _index_to_column_letter(self, index):
        """Convert column index to Excel letter (A, B, C, AA, AB, AC...)"""
        letters = ''
        while index >= 0:
            letters = chr(65 + (index % 26)) + letters
            index = index // 26 - 1
        return letters


print("📊 Export Status Updater ready")

In [ ]:
# Cell 5: CLI Interface - User-Guided Export Workflow

def run_export_workflow():
    """Interactive export workflow - guides user through entire process"""
    
    print("\n" + "="*60)
    print("🎯 3D DATA MANAGEMENT - EXPORT OPERATIONS")
    print("="*60)
    print("📋 Delivering staged files to client folders")
    
    try:
        # Step 1: Load configuration
        print("\n⚙️ Loading configuration...")
        config_reader = ConfigReader(SHEET_ID)
        config = config_reader.load_config()
        
        # Step 2: Get available export batches
        print("\n📖 Reading staged tasks...")
        task_reader = StagedTaskReader(SHEET_ID)
        batches = task_reader.get_export_batches()
        
        if not batches:
            print("\n❌ No staged export batches found")
            print("   Run staging operations first to prepare tasks for export")
            return
        
        # Step 3: Select export batch
        print(f"\n📦 Available export batches:")
        for i, batch in enumerate(batches, 1):
            print(f"   {i}. {batch['batch_id']} ({batch['task_count']} tasks) - {batch['created']}")
        
        while True:
            try:
                selection = input(f"\n🔍 Select batch [1-{len(batches)}]: ").strip()
                batch_index = int(selection) - 1
                if 0 <= batch_index < len(batches):
                    selected_batch = batches[batch_index]
                    break
                else:
                    print(f"Please enter a number between 1 and {len(batches)}")
            except ValueError:
                print("Please enter a valid number")
        
        print(f"✅ Selected: {selected_batch['batch_id']}")
        
        # Step 4: Get tasks for selected batch
        tasks = task_reader.get_staged_tasks(selected_batch['batch_id'])
        print(f"📋 Found {len(tasks)} staged tasks ready for export")
        
        if len(tasks) <= 5:
            for task in tasks:
                print(f"   • {task['folder_name']}")
        else:
            for task in tasks[:3]:
                print(f"   • {task['folder_name']}")
            print(f"   ... and {len(tasks) - 3} more")
        
        # Step 5: Get client folder ID
        print("\n🎯 Client folder setup:")
        client_folder_id = input("   Enter client Google Drive folder ID: ").strip()
        
        if not client_folder_id:
            print("❌ Client folder ID is required")
            return
        
        # Step 6: Validate client folder access
        print("\n🔍 Validating client folder...")
        export_ops = ExportOperations(config['STAGING_FOLDER_ID'])
        validation = export_ops.validate_client_folder(client_folder_id)
        
        if not validation['valid']:
            print(f"❌ Client folder validation failed: {validation['error']}")
            return
        
        print(f"✅ Client folder validated: {validation['name']}")
        
        # Step 7: Final confirmation
        print(f"\n🚀 Ready to export:")
        print(f"   📦 Batch: {selected_batch['batch_id']}")
        print(f"   📁 Tasks: {len(tasks)}")
        print(f"   🎯 Client: {validation['name']}")
        
        confirm = input(f"\n✋ Proceed with export? [y/N]: ").strip().lower()
        if confirm != 'y':
            print("❌ Export cancelled")
            return
        
        # Step 8: Execute export with progress tracking
        print(f"\n🔥 Starting export operation...")
        start_time = time.time()
        
        export_results = export_ops.export_tasks_concurrent(
            tasks, 
            client_folder_id,
            max_workers=3  # Conservative for stability
        )
        
        # Step 9: Update sheet with results
        print("\n📊 Updating sheet status...")
        status_updater = ExportStatusUpdater(SHEET_ID)
        status_updater.update_delivery_status(export_results)
        
        # Step 10: Final summary
        duration = time.time() - start_time
        successful = sum(1 for r in export_results if r['success'])
        failed = len(export_results) - successful
        
        print("\n" + "="*60)
        print("🎉 EXPORT COMPLETE")
        print("="*60)
        print(f"📦 Batch: {selected_batch['batch_id']}")
        print(f"🎯 Client: {validation['name']}")
        print(f"✅ Delivered: {successful}")
        print(f"❌ Failed: {failed}")
        print(f"⏱️ Duration: {duration:.1f}s")
        
        if failed > 0:
            print("\n❌ Failed tasks:")
            for result in export_results:
                if not result['success']:
                    print(f"   • {result['folder_name']}: {result['error']}")
        
        print("\n🏁 Export operation completed!")
        
    except Exception as e:
        print(f"\n💥 Export failed: {str(e)}")
        raise


# Export workflow ready
print("\n🎯 Export Workflow Ready!")
print("📞 Run: run_export_workflow()")
print("\n💡 Make sure you have:")
print("   • Staged tasks ready for export")
print("   • Client Google Drive folder ID")
print("   • Write access to client folder")